In [ ]:
from glob import glob
from PIL import Image
import numpy as np
import os
import sys

In [ ]:
image_files = ( glob(f'StockPhotos/*.{ext}') for ext in ('png','jpg','jpeg') )
image_files = [ name for names in image_files for name in names ]

for image_file in image_files:
    image = Image.open(image_file)
    print(f'In: {image_file}')
    
    bbox = image.getbbox()
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    
    bands = list(image.getbands())
    nbands = len(bands)
    if nbands % 2 == 0:
        nbands = nbands - 1 
        bands = bands[0:nbands]
        
    if nbands == 3:
        (r,g,b) = tuple( np.array(image.getdata(i),dtype='B') for i in range(nbands) )
        deltas = np.abs(r - g) + np.abs(r - b) + np.abs(g-b)
        if deltas.max() > 0:
            print(f'RGB Image {w}x{h}')
        else:
            print(f'Flatten RGB to Monotone Image {w}x{h}')
            nbands = 1
            z = r
    else:
        print(f'Monotone Image {w}x{h}')
        z = np.array(image.getdata(0),dtype='B')
        
    raster_filename = os.path.splitext(image_file)[0] + ('_bw.rd' if nbands == 1 else '_rgb.rd')
    print(f'Out: {raster_filename}')
    
    f = open(raster_filename,'wb')
    f.write(w.to_bytes(4,sys.byteorder))
    f.write(h.to_bytes(4,sys.byteorder))
    f.write(nbands.to_bytes(4,sys.byteorder))
    
    if nbands == 3:
        f.write(r.tobytes())
        f.write(g.tobytes())
        f.write(b.tobytes())
    else:
        f.write(z.tobytes())
    
    f.close()
    
    display(image)